In [2]:
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt

df=pd.read_csv("other_data.csv")
df.type.replace(df.type.unique(),np.array([i for i in range(23)]),inplace=True)#remplacer les types d'images par 
                                                                               #des valeurs numérique (0 à 22)
pors=df.type.value_counts().apply(lambda x : x*20//100)#calculé le nombre d'images a extraire de chaque type pour 20%


In [2]:
# importer les données dans des listes img_test et img_train
list_img_x_test=[]
list_img_x_train=[]
list_img_y_test=[]
list_img_y_train=[]

for ty in df.type.unique():
    list_img_x_test.extend(df.filename[df.type==ty].sample(n=pors[ty]))#la fonction sample pérmer d'extrait des lignes
                                                                       #d'un DataFrame d'une maniere aleatoire
    list_img_y_test.extend([ty]*pors[ty])
for i,j in zip(df.filename,df.type):
    if i not in list_img_x_test:
        list_img_x_train.append(i)
        list_img_y_train.append(j)

img_test=[]
img_train=[]

for img_n  in list_img_x_train:
    img=cv.imread('final/'+img_n)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_train.append(img)
    
for img_n  in list_img_x_test:
    img=cv.imread('final/'+img_n)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_test.append(gray)

In [ ]:
# transformer les images en textes et les sauvgarder dans des listes x_train_text, x_test_text
import matplotlib.pyplot as plt
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
x_train_text=[]
x_test_text=[]
for img in img_train:
    text=pytesseract.image_to_string(img)
    x_train_text.append(text)


In [52]:
for img in img_test:
    text=pytesseract.image_to_string(img)
    x_test_text.append(text)

In [53]:
#Convertissez une collection de documents texte en une matrice de décomptes de jetons
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x_train_text)


https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#from-occurrences-to-frequencies

(500, 10990)

In [54]:
 tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [59]:
#classificateur Bayes naïf 
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [126]:
#classificateur(SVM)
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
))])

In [127]:
#LogisticRegression
from sklearn import ensemble
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', linear_model.LogisticRegression(C=1000)
)])

In [128]:
text_clf.fit(x_train_text, list_img_y_train)

C:\Users\sadoun\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1000))])

In [129]:
y_pred=text_clf.predict(x_test_text)
text_clf.score(x_test_text,list_img_y_test)


0.6089743589743589

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ac=AdaBoostClassifier(base_estimator=clf,n_estimators=400)
ac.fit(x_train, y_train)
ac.score(x_test,y_test)